# What is object-oriented programming?
Say you want to calculate the area of a rectangle ($A = wl$, where $w$ is the width and $l$ is the length). Intuitively, you may do something like this:

In [5]:
def rectangle_area(width, length):
    return width * length

This means, you would simply write a function that takes two numbers and returns the result. While this is a great way to do things for some tasks, more complex programming often becomes cumbersome this way. But it does not have to be this way. Some very smart people (see e.g. [here](http://web.eecs.utk.edu/~huangj/CS302S04/notes/oo-intro.html)) came up with the idea of classes and objects. Let's see what that means:

In [6]:
class Rectangle:
    def __init__(self, width, length):
        self.width = width
        self.length = length
    
    def area(self):
        return self.width * self.length

Before explaining in detail what is happening here, let's see that these two things do exactly the same:

In [7]:
width = 3
length = 5

# Function evaluation
print(f'Function version: {rectangle_area(width, length)}')

# Object-based version:
the_rectangle = Rectangle(width, length)
print(f'Object-based version: {the_rectangle.area()}')


Function version: 15
Object-based version: 15


So what are we doing here? Instead of simply writing down an equation, like in `rectangle_area`, we create an **instance** or a specific rectangle, which has a given width and length. And then we use these **properties** to evaluate the area.
+ This means if you want to "do" a lot of things to this rectangle, you can have all of this information contained in one class. This makes it safe and easily readable.
+ You can use classes in other classes. So for example you could make a `Point` class, which is then used in a `Line` class, which is then used in a `Triangle` class etc. This makes for intuitively readable code.

# Linking optimization and object-oriented programming
But how does this relate to optimization? Well, I'm going to show two ways here:

## Indexing
The first one is indexing. Optimization is choosing the best feasible values for our degrees of freedom based on a metric described by an objective function. Thus, lists and indexing always surround optimization models. Let's look at a simple minimum spanning tree model:
\begin{equation}
\begin{array}{ll}
\underset{x}{\text{minimize}} & \sum \limits_{o,d} c_{o,d}x_{o,d} \\
\text{subject to} & \sum \limits_d x_{o,d} = 1, \hspace{0.3cm} \forall o \in V \\
& x_{o,d} \in \{0,1\}
\end{array}
\end{equation}

Before getting started, let's generate some random test data. We'll be using Xpress for the solution.

In [10]:
import xpress as xp
import numpy as np
from scipy import spatial

n = 500 # Number of points
V = 1e3 * np.random.rand(n,2) # Generate points and scale them.
range_V = range(n)
distances = spatial.distance.cdist(V,V)

Intuitively, you may think of doing something like this:

In [12]:
# Generate the model object
mdl = xp.problem("Standard indexing")

# Create our variables and add them to the model
x = {(o,d) : xp.var(vartype = xp.binary, lb = 0, ub = 1, name = f'x_{o},{d}') 
     for o in range_V for d in range_V if d != o}
mdl.addVariable(x)

# Create objective function - based on distance
mdl.setObjective(xp.Sum(distances[o,d]*x[o,d] for o in range_V for d in range_V if d != o))

# Set the constraint
MinimumSpanningTree = [xp.constraint(xp.Sum(x[o,d] for d in range_V if d != o) == 1, 
                                     name = f'Enforce connection for x_{o}') for o in range_V]
mdl.addConstraint(MinimumSpanningTree)

mdl.solve()

Here, we simply create our indices and run through them as need be. However, we can also see that we always need to include a `d != o` in any equation to make it work. Sure, we could define a list of tuples `(o,d)` and use that instead. But then let's imagine we want to change the cost function to have an individual cost for each connection - maybe a geographical feature of the physical system we are modelling. What then?

Here, object-oriented programming comes to the rescue. Let's implement these two things that were mentioned and see how that looks:

In [13]:
# Define the class used
class Connection:
    def __init__(self, origin, destination, cost_per_connection):
        self.origin = origin
        self.destination = destination
        self.cost = cost_per_connection
        
# Create list of connections
costs = np.random.rand(n,n)
list_of_connections = [Connection(o,d,costs[o,d]) for o in range_V for d in range_V if d != o]
        
# Generate the model object
mdl = xp.problem("Object-oriented approach")

# Create our variables and add them to the model
x = {connection : xp.var(vartype = xp.binary, lb = 0, ub = 1, 
                         name = f'x_{connection.origin},{connection.destination}') 
     for connection in list_of_connections}
mdl.addVariable(x)

# Create objective function - based on distance
mdl.setObjective(xp.Sum(connection.cost*x[connection] for connection in list_of_connections))

# Set the constraint
MinimumSpanningTree = [xp.constraint(xp.Sum(x[c] for c in list_of_connections if c.origin == o) == 1, 
                                     name = f'Enforce connection for x_{o}') for o in range_V]
mdl.addConstraint(MinimumSpanningTree)

mdl.solve()

At first glance, this may look a bit clunky. However, consider e.g. the objective function: instead of drawing on another variable, which is defined somewhere else and somewhat "floating" around, we now have a tight connection between the variable object `x` and the thing it is meant to represent. Need to change the cost for a specific connection? No problem. Need to make this a capacitated problem? Simply add a `number_of_attached_nodes` property and describe a constraint with it. The possibilities are truly endless.

## Model building
When I look around the internet (e.g. the [git repo for docplex](https://github.com/IBMDecisionOptimization/docplex-examples/)), I often see code along the following lines:

In [1]:
def Model(input_data):
    mdl = create_model_object()
    x = create_variables()
    
    mdl = create_constraint_1(mdl, x, input_data)
    mdl = create_constraint_2(mdl, x, input_data)
    
    mdl = set_objective_function(mdl, x, input_data)
    
    mdl.solve()
    
    return mdl    

In other words, it simply takes some input data, and then, confined within a bunch of functions, we create our model. This is technically totally fine, and it works. However, if you look at the code above, it does seem like we are repeating `mdl`, `x` and `input_data` quite a bit. If you then imagine something slightly more complicated, with e.g. some callbacks and conditionality, this very quickly blows up to be a 1500 line monster that you never intended for.

So what to do? Well, again object orientation comes to the rescue. And I would even argue that it is more impactful here than in the indexing case, because it impacts a larger part of the codebase. Let's try to rewrite this example from above in object-oriented code:

In [6]:
class Modeller:
    # Here we define the properties of our modeller class
    def __init__(self, input_data):
        self.input_data = input_data
        self.mdl = create_model_object()
        self.x = create_variables()
        self.objective = None
    
    def create_constraint_1(self):
        # The content of the `create_constraint_1` function of before, but using the object
        # properties self.input_data, self.mdl and self.x instead
        self.mdl
        
    def create_constraint_2(self):
        # As above
        self.mdl
        
    def set_objective_function(self):
        # As above
        self.obective
        
    def model_creation_and_solution(self):
        self.create_constraint_1()
        self.create_constraint_2()
        self.set_objective_function()
        
        self.mdl.solve()

While this again may look longer than the functional programming counterpart, there are some clear advantages to this: first, you minimize the amount of **replicate calls** to `mdl`, `x` and `input_data`.

Second, your object is now **responsible** for these properties (see the [SOLID](https://en.wikipedia.org/wiki/SOLID) pricinples for this), so when you write your constraints you don't have to worry about what goes in and out.

Third, this is much more **testable**: while testing is not really easily done with optimization code (I may write something about that at some point), it is essential programming practice. Each block in this `Modeller` class is easily testable. Now you might say that the functional programming equivalent is also easily testable, and that is true to an extent, however it comes much more natural to this object-oriented setup.

Lastly, it also **represents more accuratly** what you really want to do: in the object-orientation, you create your variables and model object and then say "right, let's take this model and add some constraints" or "let's add an objective function". In the functional programming way, you pass the entire model object back and forth for a simple constraint. That just does not feel right.

# In conclusion
I've written optimization applications in MATLAB, Python and C#, and I used to be really annoyed by the whole index tracking, needing to keep passing data back and forth. However, since I started being disciplined about using object-oriented programming, my code has become so much clearer and cleaner just with these two rules:
1. Use objects to index when possible
2. Create a Modeller class with the model object, model variables and objective function as properties and the constraints as methods.

What do you think? Do you have any experience with object orientation in mathematical optimization?